# GenZ Career Aspirations Dashboard

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json

# Load the data
df =pd.read_csv("data/GenZ.csv")


In [1]:
# Original column names
original_columns = [
    'Your Current Country.',
    'Your Current Zip Code / Pin Code',
    'Your Gender',
    'Which of the below factors influence the most about your career aspirations ?',
    'Would you definitely pursue a Higher Education / Post Graduation outside of India ? If only you have to self sponsor it.',
    'How likely is that you will work for one employer for 3 years or more ?',
    'Would you work for a company whose mission is not clearly defined and publicly posted.',
    'How likely would you work for a company whose mission is misaligned with their public actions or even their product ?',
    'How likely would you work for a company whose mission is not bringing social impact ?',
    'What is the most preferred working environment for you.',
    'Which of the below Employers would you work with.',
    'Which type of learning environment that you are most likely to work in ?',
    'Which of the below careers looks close to your Aspirational job ?',
    'What type of Manager would you work without looking into your watch ?',
    'Which of the following setup you would like to work ?'
]

# New short names
new_columns = ['country', 'zip_code', 'gender', 'career_factors', 'higher_ed_abroad', 
               'long_term_employer', 'unclear_mission', 'misaligned_mission', 
               'no_social_impact', 'work_env', 'employer_choice', 'learning_env', 
               'asp_job', 'manager_type', 'work_setup']

# Create col_map (short name -> original name)
col_map = dict(zip(new_columns, original_columns))

# Rename the columns
df.columns = new_columns

# Optional: Save col_map to CSV
col_map_df = pd.DataFrame(list(col_map.items()), columns=['Short_Name', 'Original_Name'])
col_map_df.to_csv('column_mapping.csv', index=False)

print("Columns renamed and mapping saved to 'column_mapping.csv'")

NameError: name 'df' is not defined

In [ ]:
df = df[df['country'] == 'India'].reset_index(drop=True)

In [ ]:
# List of questions (columns) for Selection Box 1
questions = {
    'career_factors': 'Which factors influence your career aspirations most?',
    'higher_ed_abroad': 'Would you pursue higher education abroad if self-sponsored?',
    'long_term_employer': 'How likely to work for one employer for 3+ years?',
    'unclear_mission': 'Would you work for a company with an unclear mission?',
    'misaligned_mission': 'How likely to work for a company with a misaligned mission?',
    'no_social_impact': 'How likely to work for a company with no social impact?',
    'work_env': 'What is your most preferred working environment?',
    'employer_choice': 'Which employers would you work with?',
    'learning_env': 'Which learning environment do you prefer?',
    'asp_job': 'Which career is closest to your aspirational job?',
    'manager_type': 'What type of manager would you work for without watching the clock?',
    'work_setup': 'Which work setup do you prefer?'
}

# HTML content with single pie chart, legend below, and fixed tab logic
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Survey Dashboard</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f4f4f9;
        }}
        .container {{
            max-width: 1200px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }}
        h1 {{
            text-align: center;
            color: #333;
            margin-bottom: 20px;
        }}
        .controls {{
            display: flex;
            justify-content: space-between;
            margin-bottom: 20px;
            gap: 20px;
        }}
        select {{
            padding: 8px;
            font-size: 14px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }}
        .chart-container {{
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
        }}
        .chart {{
            width: 60%;
        }}
        .tabs {{
            margin-bottom: 20px;
        }}
        .tab {{
            padding: 10px 20px;
            cursor: pointer;
            display: inline-block;
            background: #ddd;
            border-radius: 5px 5px 0 0;
        }}
        .tab.active {{
            background: #007bff;
            color: white;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1>Survey Insights Dashboard</h1>
        <div class="tabs">
            <span class="tab active" onclick="setView('top'); updateChart();">Top N Responses</span>
            <span class="tab" onclick="setView('bottom'); updateChart();">Bottom N Responses</span>
        </div>
        <div class="controls">
            <div>
                <label for="question">Question :</label>
                <select id="question" onchange="updateChart()">
                    {question_options}
                </select>
            </div>
            <div>
                <label for="gender">Gender :</label>
                <select id="gender" onchange="updateChart()">
                    <option value="All">All</option>
                    <option value="Male">Men</option>
                    <option value="Female">Women</option>
                </select>
            </div>
            <div>
                <label for="nResponses"> N :</label>
                <select id="nResponses" onchange="updateChart()">
                    <option value="3" selected>3</option>
                    <option value="5">5</option>
                </select>
            </div>
        </div>
        <div class="chart-container">
            <div id="chart" class="chart"></div>
        </div>
    </div>

    <script>
        const df = {df_json};
        let currentView = 'top';

        // Populate question dropdown
        const questionOptions = `{question_options}`.replace(/<option value="/g, '<option value=\\"').replace(/">/g, '\\">');
        document.getElementById('question').innerHTML = questionOptions;

        // Function to set the active tab
        function setView(view) {{
            currentView = view;
            const tabs = document.querySelectorAll('.tab');
            tabs.forEach(tab => {{
                tab.classList.remove('active');
                if (tab.textContent === (view === 'top' ? 'Top Responses' : 'Bottom Responses')) {{
                    tab.classList.add('active');
                }}
            }});
        }}

        function updateChart() {{
            const question = document.getElementById('question').value;
            const gender = document.getElementById('gender').value;
            const n = parseInt(document.getElementById('nResponses').value);

            let filteredDf = df;
            if (gender !== 'All') {{
                filteredDf = filteredDf.filter(row => row.gender === gender);
            }}

            const counts = {{}};
            filteredDf.forEach(row => {{
                const value = row[question];
                counts[value] = (counts[value] || 0) + 1;
            }});

            const sortedCounts = Object.entries(counts)
                .map(([key, value]) => ({{name: key, count: value}}))
                .sort((a, b) => currentView === 'top' ? b.count - a.count : a.count - b.count);

            // Select top N or bottom N based on currentView
            const data = currentView === 'top' ? 
                sortedCounts.slice(0, n) : 
                sortedCounts.slice(-n);

            const chartData = [{{
                type: 'pie',
                labels: data.map(d => d.name),
                values: data.map(d => d.count),
                textinfo: 'percent',
                textposition: 'inside'
            }}];

            const layout = {{
                title: currentView === 'top' ? 'Top ' + n + ' Responses' : 'Bottom ' + n + ' Responses',
                legend: {{x: 0.5, y: -0.1, xanchor: 'center', yanchor: 'top', orientation: 'h'}},
                margin: {{t: 50, b: 100, l: 50, r: 50}}
            }};

            Plotly.newPlot('chart', chartData, layout);
        }}

        // Initial chart render
        updateChart();
    </script>
</body>
</html>
"""

# Generate question options for dropdown
question_options = ''.join([f'<option value="{key}">{value}</option>' for key, value in questions.items()])

# Convert DataFrame to JSON for embedding
df_json = df.to_json(orient='records')

# Insert dynamic content into HTML
html_content = html_content.format(
    question_options=question_options,
    df_json=df_json
)

# Write to file
with open('survey_dashboard.html', 'w') as f:
    f.write(html_content)

print("Dashboard saved as 'survey_dashboard.html'")

Dashboard saved as 'survey_dashboard.html'


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json


questions = {
    'career_factors': 'Which factors influence your career aspirations most?',
    'higher_ed_abroad': 'Would you pursue higher education abroad if self-sponsored?',
    'long_term_employer': 'How likely to work for one employer for 3+ years?',
    'unclear_mission': 'Would you work for a company with an unclear mission?',
    'misaligned_mission': 'How likely to work for a company with a misaligned mission?',
    'no_social_impact': 'How likely to work for a company with no social impact?',
    'work_env': 'What is your most preferred working environment?',
    'employer_choice': 'Which employers would you work with?',
    'learning_env': 'Which learning environment do you prefer?',
    'asp_job': 'Which career is closest to your aspirational job?',
    'manager_type': 'What type of manager would you work for without watching the clock?',
    'work_setup': 'Which work setup do you prefer?'
}

# HTML content with styled title, single pie chart, and legend below
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Survey Dashboard</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f4f4f9;
        }}
        .container {{
            max-width: 1200px;
            margin: auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }}
        .controls {{
            display: flex;
            justify-content: space-between;
            margin-bottom: 20px;
            gap: 20px;
        }}
        select {{
            padding: 8px;
            font-size: 14px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }}
        .chart-container {{
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
        }}
        .chart {{
            width: 60%;
        }}
        .tabs {{
            margin-bottom: 20px;
        }}
        .tab {{
            padding: 10px 20px;
            cursor: pointer;
            display: inline-block;
            background: #ddd;
            border-radius: 5px 5px 0 0;
        }}
        .tab.active {{
            background: #007bff;
            color: white;
        }}
        @keyframes fadeIn {{
            from {{ opacity: 0; transform: translateY(-10px); }}
            to {{ opacity: 1; transform: translateY(0); }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div style="background: linear-gradient(90deg, #00B8D9, #00ACC1); padding: 40px 20px; border-radius: 15px; text-align: center; animation: fadeIn 2s ease-in-out;">
            <h1 style="color: white; font-size: 48px; margin-bottom: 10px;">Career Preferences Report: Gen Z Edition</h1>
            <h3 style="color: #ECEFF1; font-weight: normal; font-size: 20px;">
                Insights into Aspirations, Motivations, Industry Choices, and Challenges Shaping the Future of Work
            </h3>
        </div>
        <div class="tabs">
            <span class="tab active" onclick="setView('top'); updateChart();">Top Responses</span>
            <span class="tab" onclick="setView('bottom'); updateChart();">Bottom Responses</span>
        </div>
        <div class="controls">
            <div>
                <label for="question">Select Question:</label>
                <select id="question" onchange="updateChart()">
                    {question_options}
                </select>
            </div>
            <div>
                <label for="gender">Gender Filter:</label>
                <select id="gender" onchange="updateChart()">
                    <option value="All">All</option>
                    <option value="Male">Men</option>
                    <option value="Female">Women</option>
                </select>
            </div>
            <div>
                <label for="nResponses">Top/Bottom N:</label>
                <select id="nResponses" onchange="updateChart()">
                    <option value="3" selected>3</option>
                    <option value="5">5</option>
                </select>
            </div>
        </div>
        <div class="chart-container">
            <div id="chart" class="chart"></div>
        </div>
    </div>

    <script>
        const df = {df_json};
        let currentView = 'top';

        // Populate question dropdown
        const questionOptions = `{question_options}`.replace(/<option value="/g, '<option value=\\"').replace(/">/g, '\\">');
        document.getElementById('question').innerHTML = questionOptions;

        // Function to set the active tab
        function setView(view) {{
            currentView = view;
            const tabs = document.querySelectorAll('.tab');
            tabs.forEach(tab => {{
                tab.classList.remove('active');
                if (tab.textContent === (view === 'top' ? 'Top Responses' : 'Bottom Responses')) {{
                    tab.classList.add('active');
                }}
            }});
        }}

        function updateChart() {{
            const question = document.getElementById('question').value;
            const gender = document.getElementById('gender').value;
            const n = parseInt(document.getElementById('nResponses').value);

            let filteredDf = df;
            if (gender !== 'All') {{
                filteredDf = filteredDf.filter(row => row.gender === gender);
            }}

            const counts = {{}};
            filteredDf.forEach(row => {{
                const value = row[question];
                counts[value] = (counts[value] || 0) + 1;
            }});

            const sortedCounts = Object.entries(counts)
                .map(([key, value]) => ({{name: key, count: value}}))
                .sort((a, b) => currentView === 'top' ? b.count - a.count : a.count - b.count);

            // Select top N or bottom N based on currentView
            const data = currentView === 'top' ? 
                sortedCounts.slice(0, n) : 
                sortedCounts.slice(-n);

            const chartData = [{{
                type: 'pie',
                labels: data.map(d => d.name),
                values: data.map(d => d.count),
                textinfo: 'percent',
                textposition: 'inside'
            }}];

            const layout = {{
                title: currentView === 'top' ? 'Top ' + n + ' Responses' : 'Bottom ' + n + ' Responses',
                legend: {{x: 0.5, y: -0.1, xanchor: 'center', yanchor: 'top', orientation: 'h'}},
                margin: {{t: 50, b: 100, l: 50, r: 50}}
            }};

            Plotly.newPlot('chart', chartData, layout);
        }}

        // Initial chart render
        updateChart();
    </script>
</body>
</html>
"""

# Generate question options for dropdown
question_options = ''.join([f'<option value="{key}">{value}</option>' for key, value in questions.items()])

# Convert DataFrame to JSON for embedding
df_json = df.to_json(orient='records')

# Insert dynamic content into HTML
html_content = html_content.format(
    question_options=question_options,
    df_json=df_json
)

# Write to file
with open('survey_dashboard.html', 'w') as f:
    f.write(html_content)

print("Dashboard saved as 'survey_dashboard.html'")

Dashboard saved as 'survey_dashboard.html'
